# Calling D (dlang) shared libs from Python

The D programming language is attractive for making native extensions
for Python, for several reasons:

- Syntax features of D are excellent
- Very fast compilation speed

But the primary benefit for this use case is build native extensions
in a modern systems programming language (i.e., not C), and to use
that same binary across multiple versions of Python. Building a 
binary once is very attractive! Currently with Cython, for example, the
extension module must be built on the target platform for *each version
of Python* that is supported. Here, we're trying to make it so that
it would be possible to build the native extension without even 
requiring Python to be installed on the build machine. (Of course, 
Python would still be required to run tests and verify binary).

The idea is similar (in the basic idea) to how the 
[milksnake](https://github.com/getsentry/milksnake) project 
uses CFFI to wrap rust-produced binaries in Python.

## The Goal

Our goal is to run a *Python* program that uses CFFI to call a 
function inside a native extension written in D.

Remember that you must install CFFI:

```shell
$ pip install cffi
```

The Python file below will use the ABI mode of loading the dll:

```python
# blah.py
from cffi import FFI

ffi = FFI()
ffi.cdef("int foo();")
C = ffi.dlopen("blah.dll")
x = C.foo()
print(f"Called the dll, and the result was {x}")
```

Running the file must produce the following:

```
$ python blah.py
Called the dll, and the result was 123 
```

This uses the less-work _ABI mode_ of CFFI.

## Windows

On Windows it's a bit odd because you need some entrypoint code
inside the shared library. This is just boilerplate though. Focus
on the actual exported functions at the bottom:

```d
// blah.d
import core.sys.windows.windows;
import core.sys.windows.dll;
import core.stdc.stdio;

__gshared HINSTANCE g_hInst;

extern (Windows)
BOOL DllMain(HINSTANCE hInstance, ULONG ulReason, LPVOID pvReserved)
{
    switch (ulReason)
    {
	case DLL_PROCESS_ATTACH:
	    g_hInst = hInstance;
	    dll_process_attach( hInstance, true );
	    break;

	case DLL_PROCESS_DETACH:
	    dll_process_detach( hInstance, true );
	    break;

	case DLL_THREAD_ATTACH:
	    dll_thread_attach( true, true );
	    break;

	case DLL_THREAD_DETACH:
	    dll_thread_detach( true, true );
	    break;

        default:
    }
    return true;
}

extern (C) int foo() {
    return 123;
}

```

You will also need a `blah.def` file:

```
LIBRARY blah

CODE SHARED EXECUTE
DATA WRITE

EXPORTS
    foo
```

To build the `.dll`, run this:

```
$ dmd -m64 -betterC -shared -ofblah.dll blah.d blah.def
```

- On Windows you must supply `-m64` to make a 64-bit shared library. This
is important if you want to load it into a 64-bit Python application.
- We're passing the `-betterC` flag to indicate we don't need the D 
standard library.
- Obviously `-shared` is necessary to make a shared library.

After building, you can inspect the DLL:

```
$ ls -lah blah.*
-rw-r--r-- 1 caleb 197121  697 May  5 12:01 blah.d
-rw-r--r-- 1 caleb 197121   69 May  5 12:06 blah.def
-rwxr-xr-x 1 caleb 197121 401K May  5 12:06 blah.dll
-rw-r--r-- 1 caleb 197121  787 May  5 12:06 blah.exp
-rw-r--r-- 1 caleb 197121 1.7K May  5 11:59 blah.lib
-rw-r--r-- 1 caleb 197121 1.8K May  5 12:06 blah.obj
```

Running the Python program `blah.py` now produces the expected output.

### Runtime requirements

On Windows, if you build your DLL with Visual studio installed, the C runtime
(`msvcrt100.dll`, first introduced with Visual Studio 2010) will be
statically linked and so the only runtime dependency, besides whatever
you explicitly require in your D code, is `kernel32.dll` which every
version of Windows back to XP will have available:

```
Setting environment for using Microsoft Visual Studio 2008 x64 tools.

G:\Documents\repos\dasbcpy>dumpbin /dependents blah.dll
Microsoft (R) COFF/PE Dumper Version 9.00.30729.01
Copyright (C) Microsoft Corporation.  All rights reserved.

Dump of file blah.dll

File Type: DLL

  Image has the following dependencies:

    KERNEL32.dll

  Summary

        1000 ._deh
        F000 .data
        1000 .dp
        1000 .minfo
        4000 .pdata
        F000 .rdata
        2000 .reloc
       44000 .text
        1000 .tp

```

## Linux

On linux, some things are easier and some things are harder. The code for the
shared library itself is quite a bit simpler:

```
// blah.d
import core.stdc.stdio;

extern (C) int foo() {
    return 123;
}
```

No special entrypoint declarations are necessary. First, building the shared 
library:

```
$ dmd -fPIC -betterC -shared -ofblah.so blah.d
```

Note that we've added the `-fPIC` option, and changed the name of the 
shared library to `blah.so`.  After changing `blah.dll` to `blah.so`
in the `blah.py` Python file, we can once again confirm that it works:

```
$ python blah.py
Called the dll, and the result was 123
```

Unfortunately, the linker links against whatever version of the C 
runtime is currently installed:

```
$ ldd blah.so
        linux-vdso.so.1 (0x00007ffc5f333000)
        libpthread.so.0 => /lib/x86_64-linux-gnu/libpthread.so.0 (0x00007f836e25e000)
        libm.so.6 => /lib/x86_64-linux-gnu/libm.so.6 (0x00007f836df5a000)
        librt.so.1 => /lib/x86_64-linux-gnu/librt.so.1 (0x00007f836dd52000)
        libdl.so.2 => /lib/x86_64-linux-gnu/libdl.so.2 (0x00007f836db4e000)
        libc.so.6 => /lib/x86_64-linux-gnu/libc.so.6 (0x00007f836d7af000)
        /lib64/ld-linux-x86-64.so.2 (0x00007f836e67d000)
```

This means that the `blah.so` will *not work* on older versions of Linux
where the the C runtime is an older version. One way to get around this
is to compile the shared library on an older version of linux, a task
which docker makes relatively easy; but it's still a pain.
